In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import my_data_frequency_conversor as dfc
import my_models as mm
import my_llm_ts_preprocessor as tsp
import my_prompting_tool as mpt 

In [11]:
# crear un diccionario con clave dataset y valor datos de training
datasets = {
    'ETH_HOURS':'DATA/ETHUSD-H1.csv',
    'ETH_DAYS':'DATA/ETHUSD-D1.csv',
    'ETH_MONTHS':'DATA/ETHUSD-Monthly.csv'
}
freqs = {
    'ETH_HOURS':'h',
    'ETH_DAYS':'d',
    'ETH_MONTHS':'MS'
}
train_data = {}
val_data = {}

In [12]:
# Itera sobre cada conjunto de datos en el diccionario 'datasets'
for ds_name in datasets:  
    # Carga el archivo CSV correspondiente en un DataFrame de pandas
    df = pd.read_csv(datasets[ds_name])  
    
    # Divide los datos en entrenamiento y validación usando la función split_data de dfc
    train_data[ds_name], val_data[ds_name] = dfc.split_data(df)  
    
    # Ajusta la frecuencia de los datos de entrenamiento según el diccionario 'freqs'
    train_data[ds_name] = train_data[ds_name].asfreq(freqs[ds_name])  
    
    # Ajusta la frecuencia de los datos de validación según el diccionario 'freqs'
    val_data[ds_name] = val_data[ds_name].asfreq(freqs[ds_name])  
    
    # Redondea los valores de los datos de entrenamiento a 3 decimales
    train_data[ds_name] = train_data[ds_name].round(3)  
    
    # Redondea los valores de los datos de validación a 3 decimales
    val_data[ds_name] = val_data[ds_name].round(3)  
    
    # Imprime la forma (cantidad de filas y columnas) del conjunto de entrenamiento
    print(f'{ds_name} train data shape: {train_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de entrenamiento
    print(train_data[ds_name].head())  
    
    # Imprime la forma del conjunto de validación
    print(f'{ds_name} val data shape: {val_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de validación
    print(val_data[ds_name].head())  


ETH_HOURS train data shape: (61344, 1)
                         y
DateTime                  
2017-01-01 00:00:00  8.044
2017-01-01 01:00:00  8.199
2017-01-01 02:00:00  8.260
2017-01-01 03:00:00  8.423
2017-01-01 04:00:00  8.570
ETH_HOURS val data shape: (8784, 1)
                          y
DateTime                   
2024-01-01 00:00:00  2286.5
2024-01-01 01:00:00  2295.6
2024-01-01 02:00:00  2305.4
2024-01-01 03:00:00  2295.8
2024-01-01 04:00:00  2275.4
ETH_DAYS train data shape: (2556, 1)
                 y
DateTime          
2017-01-01   8.044
2017-01-02   8.220
2017-01-03   8.370
2017-01-04   9.662
2017-01-05  11.062
ETH_DAYS val data shape: (366, 1)
                 y
DateTime          
2024-01-01  2286.5
2024-01-02  2352.7
2024-01-03  2358.2
2024-01-04  2211.3
2024-01-05  2271.0
ETH_MONTHS train data shape: (84, 1)
                 y
DateTime          
2017-01-01   8.044
2017-02-01  10.711
2017-03-01  15.830
2017-04-01  49.777
2017-05-01  83.518
ETH_MONTHS val data shape: (12, 1

In [ ]:
for dataset in datasets:
    print("-" * 50)  # Línea divisoria
    print(f'Processing {dataset} dataset')
    train, test, seq = mpt.paraphrase_nlp(dataset, train_data[dataset], val_data[dataset])




--------------------------------------------------
Processing ETH_HOURS dataset
                         y
DateTime                  
2017-01-01 00:00:00  8.044
2017-01-01 01:00:00  8.199
2017-01-01 02:00:00  8.260
2017-01-01 03:00:00  8.423
2017-01-01 04:00:00  8.570
                          y
DateTime                   
2024-01-01 00:00:00  2286.5
2024-01-01 01:00:00  2295.6
2024-01-01 02:00:00  2305.4
2024-01-01 03:00:00  2295.8
2024-01-01 04:00:00  2275.4


KeyError: 'E'

In [8]:
print(seq)

0     2286.5
1     2283.0
2     3339.2
3     3650.6
4     3020.3
5     3768.2
6     3441.7
7     3237.0
8     2519.4
9     2607.8
10    2519.0
11    3708.4
dtype: float64


In [ ]:
# contar el número de tokens de args
import tiktoken as tk

encoding = tk.get_encoding("cl100k_base")  # Compatible con GPT-4 y GPT-3.5-turbo

# Contar tokens
num_tokens = len(encoding.encode(text))
print(f"Número de tokens: {num_tokens}")